# Calculate doublet scores for each cell using Scrublet

## Import statements

In [1]:
import scanpy as sc
import scanpy.external as sce
#sc.logging.print_versions()
#sc.logging.print_memory_usage()
#sc.settings.verbosity = 2
import os,sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mticker

In [2]:
#import scrublet
import scrublet as scr

In [3]:
# add the utility function folder to PATH
sys.path.append(os.path.abspath("utility_functions_190403_12h24/"))

from rz_import_statements import *
import rz_functions as rz
import rz_fig_params # this adjust mpl.rcParams, almost nothing to import, import after scanpy to overwrite rc.Params
import rz_utility_spring as srz

python version: 3.8.8


# Load data

In [4]:
adata = sc.read_h5ad('backups_JZ_2022/kidney_v0_156_v2000_batch_corrected_51196x2000_220121_15h23.h5ad') 
# overwrite obs with the most recent version
filename = 'backups_JZ_2022/clinical_obs_info_51196x15_220120_14h54.npz'
encoding = 'latin1'

with np.load(filename,encoding=encoding, allow_pickle = True) as f:
    obs = pd.DataFrame(**f)
adata.obs = obs


## Load details on the latest SPRING plot  
In general, this is not necessary to run Scrublet.  

In [6]:
# load intermediates I saved when preparing the SPRING plot
path1 = '/Users/justina/Documents/mokslai/MAGISTRAS/MAGISTRINIS/data_and_spring/SPRING_dev-master/kidney_spring/' 
project_dir = path1+'kidney_2022/'
plot_name =  'kidney_v0_156_v2000_spring'


params = rz.load_stuff(project_dir+plot_name+'/params.pickle')
params.keys()

dict_keys(['k', 'cell_mask', 'min_counts', 'min_cells', 'base_ix', 'num_pc', 'plot_name', 'embedding', 'gene_names_excluded', 'abundant_gene_mask', 'v_score_dict', 'nr_var_genes', 'genes_used', 'eigenvectors', 'eigenvalues', 'neighbors', 'min_dist'])

In [7]:
# make sure the data is unnormalized, should sum to different total counts (and round numbers)
adata.raw.X[:5,:].sum(axis=1)

matrix([[ 449.],
        [ 449.],
        [1229.],
        [ 732.],
        [ 432.]], dtype=float32)

In [8]:
P = params['eigenvectors'][:params['num_pc'],:] #the P is the same for fa2 and umap representations
P.shape

(86, 2000)

In [9]:
bdata = adata.raw.to_adata() #it is not necessary here to create a new variable

In [10]:
bdata

AnnData object with n_obs × n_vars = 51196 × 33538
    obs: 'library', 'total_counts', 'pct_counts_mito', 'library2', 'sample', 'patient', 'pT stage', 'seq_date', 'beads', 'operation', 'sex', 'tumor size, mm', 'age', 'tissue', 'necrosis'
    uns: 'X_lin_cptt', 'X_log_z', 'beads_colors', 'draw_graph', 'neighbors', 'pca', 'sample_colors', 'seq_date_colors', 'tissue_colors', 'umap'
    obsm: 'X_draw_graph_fa', 'X_pca', 'X_pca_harmony', 'X_umap'

In [11]:

# prepare  variables to plug into scrublet
gene_mask = np.in1d(bdata.var_names,params['genes_used'])
cell_mask = params['cell_mask'].astype(bool)
n_counts = np.array(adata[cell_mask,:].raw.X.sum(axis=1).T)[0]

P = params['eigenvectors'][:params['num_pc'],:] #eigenvector
print(gene_mask.sum(),len(gene_mask))
print(cell_mask.sum(),len(cell_mask))
postnorm_total = 1e4

X = bdata[cell_mask].X
X_norm = sc.pp.normalize_per_cell(bdata[cell_mask],counts_per_cell_after=postnorm_total,copy=True).X
subobs = adata.obs[cell_mask]
X_norm.sum(axis=1)

2000 33538
51196 51196


matrix([[10000.001],
        [10000.   ],
        [10000.001],
        ...,
        [ 9999.999],
        [ 9999.999],
        [ 9999.999]], dtype=float32)

In [12]:
gene_means = X_norm[:,gene_mask].mean(0).A.squeeze()
gene_stdevs = np.sqrt(scr.sparse_var(X_norm[:, gene_mask]))

In [13]:
# double-check the shapes
print(P.shape,len(gene_means))

(86, 2000) 2000


## Calculate doublet scores

### Decide on which cells to treat collectively

In [14]:
# doublet can only be made of real cells present in the same cell suspension being injected into
# a scRNAseq device.
# therefore, it makes sense to run the doublet detector on each separate cell suspension (in other words,
# for each emulsion)
# also, the doublet rate can be different between samples.

splitby = 'library2'
subobs[splitby].unique()

array(['T2_1', 'N3_1', 'N4_1', 'N2_1', 'T3_1', 'N6_1', 'N5_1', 'N7_1',
       'N8_1', 'T5_2', 'T6_3', 'T7_3', 'T8_3', 'T5_1', 'T6_1', 'T6_2',
       'T7_2', 'T7_1', 'T8_2', 'T8_1', 'N9_1', 'T9_1', 'T9_2', 'T9_3',
       'N1_old', 'N2_old', 'N3_old', 'N4_old', 'T2_old', 'T3_old',
       'T4_old'], dtype=object)

In [15]:
subobs.groupby([splitby]).count().dropna().iloc[:,:1]

,library
library2,
N1_old,2970
N2_1,758
N2_old,1563
N3_1,245
N3_old,818
N4_1,1460
N4_old,571
N5_1,573
N6_1,802


### Run Scrubet on each emulsion separately  
By design, doublets cannot form between emulsions  

In [17]:
# I want to apply the same principal component transformation to all emulsions but allow
# simulated doublets only within the same emulsion

# This is 

scrub_dict = {}
doub_score_dict = {}
predict_doub_dict = {}

for emulsion in adata.obs[cell_mask][splitby].unique():
    cmask = (adata.obs[cell_mask][splitby] == emulsion).values
    
    print(emulsion)
    
    # select the expression data:
    E = X[cmask,:][:,gene_mask]
    
    # initiate the scrub object
    scrub = scr.Scrublet(E, total_counts=n_counts[cmask], expected_doublet_rate=0.06)
     
    # Simulate doublets
    scrub.simulate_doublets()
    
    # Total counts normalize
    scrub._E_obs_norm = scr.tot_counts_norm(scrub._E_obs, total_counts=n_counts[cmask], target_total=postnorm_total)
    scrub._E_sim_norm = scr.tot_counts_norm(scrub._E_sim, total_counts=scrub._total_counts_sim, target_total=postnorm_total)
    
    # Z-score using previously calculated gene means and standard deviations. Same way as for original spring plot.
    scrub._E_obs_norm = np.array(scr.sparse_zscore(scrub._E_obs_norm, gene_means, gene_stdevs))
    scrub._E_sim_norm = np.array(scr.sparse_zscore(scrub._E_sim_norm, gene_means, gene_stdevs))
    
    # Transform simulated doublets into same PCA space as the observed cells
    pca_obs = P.dot(scrub._E_obs_norm.T).T
    pca_sim = P.dot(scrub._E_sim_norm.T).T
    
    # Set the "manifold", i.e. the space in which the knn graph will be constructed
    scrub.set_manifold(pca_obs, pca_sim)
    
    # Calculate doublet scores and call doublets as usual
    doublet_scores = scrub.calculate_doublet_scores()
    predicted_doublets = scrub.call_doublets()
    
    # store results
    scrub_dict[emulsion] = scrub
    doub_score_dict[emulsion] = doublet_scores
    predict_doub_dict[emulsion] = predicted_doublets

T2_1
Automatically set threshold at doublet score = 0.51
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 14.3%
N3_1
Automatically set threshold at doublet score = 0.24
Detected doublet rate = 4.9%
Estimated detectable doublet fraction = 26.1%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 18.7%
N4_1
Automatically set threshold at doublet score = 0.42
Detected doublet rate = 0.8%
Estimated detectable doublet fraction = 12.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 6.9%
N2_1
Automatically set threshold at doublet score = 0.36
Detected doublet rate = 0.7%
Estimated detectable doublet fraction = 3.0%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 22.2%
T3_1
Automatically set threshold at doublet score = 0.45
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.2%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.0%
N6_1
Automatically set threshold at doub

## Check doublet histograms

In [18]:
# print the automatically determined threshold:
for key,value in scrub_dict.items():
    print(key,value.threshold_)

T2_1 0.5074762440517129
N3_1 0.2377100591715978
N4_1 0.4204785862730435
N2_1 0.3612945832402352
T3_1 0.44554143659499634
N6_1 0.36097947629453375
N5_1 0.3238098681804424
N7_1 0.3366645781555726
N8_1 0.41312677809388315
T5_2 0.38899427182285307
T6_3 0.33498924913310124
T7_3 0.4961452410413042
T8_3 0.43169983479428675
T5_1 0.5470817276617818
T6_1 0.4167034770848328
T6_2 0.5001213756352467
T7_2 0.4994429059779057
T7_1 0.4826624034130415
T8_2 0.41523134565587383
T8_1 0.5138723762057498
N9_1 0.3099023349250928
T9_1 0.4641069347911506
T9_2 0.4621029682163116
T9_3 0.47698520519330395
N1_old 0.5199170055025315
N2_old 0.4443074356707133
N3_old 0.3555576615911435
N4_old 0.31578577430196486
T2_old 0.3562201168345494
T3_old 0.3562028380855369
T4_old 0.4605355801151103


In [19]:
for emulsion, scrub in scrub_dict.items():
    print(emulsion,(adata.obs[splitby] == emulsion).sum())
    dmask = predict_doub_dict[emulsion]
    print('predicted doublet percentage',dmask.sum()/len(dmask)*100.)
    scrub.plot_histogram();
    plt.show()
    plt.savefig('%s_v0_doublet_scores_prec_pca.pdf'%emulsion)
    print()

findfont: Font family ['Myriad Pro'] not found. Falling back to DejaVu Sans.


T2_1 5186
predicted doublet percentage 0.03856536829926726


findfont: Font family ['Myriad Pro'] not found. Falling back to DejaVu Sans.
<ipython-input-19-d17f4515760b>:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()



N3_1 245
predicted doublet percentage 4.8979591836734695


<ipython-input-19-d17f4515760b>:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


...

<ipython-input-19-d17f4515760b>:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()



T3_old 1666
predicted doublet percentage 0.12004801920768307


<ipython-input-19-d17f4515760b>:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()



T4_old 1810
predicted doublet percentage 0.11049723756906078



<ipython-input-19-d17f4515760b>:6: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Add results to adata.obs

In [20]:
obs = adata.obs.copy()

# add the results to obs
obs['doublet_score'] = np.nan
obs['potential_doublet'] = np.nan

for key,value in doub_score_dict.items():
    cmask = (obs[splitby] == key).values
    
    obs.loc[cmask&cell_mask,'doublet_score'] = value
    obs.loc[cmask&cell_mask,'potential_doublet'] = predict_doub_dict[key]
    
    # also add another colortrack: within the x% with highest doublet score
    for i in [3,5,10]:   
        obs.loc[cmask&cell_mask,'top%dpct_dbtl_score'%i] = \
        obs.loc[cmask&cell_mask,'doublet_score']>=obs.loc[cmask,'doublet_score'].quantile(1.-float(i)/100.)

In [21]:
obs.head()

,library,total_counts,pct_counts_mito,library2,sample,patient,pT stage,seq_date,beads,operation,sex,"tumor size, mm",age,tissue,necrosis,doublet_score,potential_doublet,top3pct_dbtl_score,top5pct_dbtl_score,top10pct_dbtl_score
2,N14,449.0,0.668151,T2_1,T2,P2,pT3a,20_11_12,old,Open,Male,75,60,Tumor,Negative,0.026847,False,False,False,False
19,N14,449.0,4.231626,T2_1,T2,P2,pT3a,20_11_12,old,Open,Male,75,60,Tumor,Negative,0.045685,False,False,False,False
363,N14,1229.0,10.659073,T2_1,T2,P2,pT3a,20_11_12,old,Open,Male,75,60,Tumor,Negative,0.102625,False,False,False,False
413,N14,732.0,4.781421,T2_1,T2,P2,pT3a,20_11_12,old,Open,Male,75,60,Tumor,Negative,0.241935,False,False,True,True
433,N14,432.0,6.944445,T2_1,T2,P2,pT3a,20_11_12,old,Open,Male,75,60,Tumor,Negative,0.061564,False,False,False,False


In [22]:
doub_score_dict.keys()

dict_keys(['T2_1', 'N3_1', 'N4_1', 'N2_1', 'T3_1', 'N6_1', 'N5_1', 'N7_1', 'N8_1', 'T5_2', 'T6_3', 'T7_3', 'T8_3', 'T5_1', 'T6_1', 'T6_2', 'T7_2', 'T7_1', 'T8_2', 'T8_1', 'N9_1', 'T9_1', 'T9_2', 'T9_3', 'N1_old', 'N2_old', 'N3_old', 'N4_old', 'T2_old', 'T3_old', 'T4_old'])

In [23]:
len(doub_score_dict.keys())

31

In [25]:
len(adata.obs['library2'].unique())

31

## Save obs only

In [26]:
# save the update obs dataframe
# no need to save the entire adata object, counts didn't change

fname = 'backups_JZ_2022/scrub_obs_info_%dx%d_%s'%(obs.shape[0],obs.shape[1],rz.now())
print(fname)
rz.save_df(obs,fname)

backups_JZ_2022/scrub_obs_info_51196x20_220125_16h34


/Users/justina/opt/anaconda3/envs/singlec4/lib/python3.8/site-packages/numpy/lib/format.py:362: UserWarning: metadata on a dtype may be saved or ignored, but will raise if saved when read. Use another form of storage.
  d['descr'] = dtype_to_descr(array.dtype)


## Optional: add colotrack to SPRING

In [27]:
# path was specified earlier

# select to append:
toappend = ['potential_doublet',
            'top3pct_dbtl_score','top5pct_dbtl_score','top10pct_dbtl_score']

toappend_cont = ['doublet_score']

# color dictionary
cdd = srz.read_cell_groupings(project_dir+plot_name+'/categorical_coloring_data.json')
cdd = {key:value['label_colors'] for key,value in cdd.items()}

# get cell filter:
cell_ix = np.loadtxt(project_dir+plot_name+'/cell_filter.txt',dtype=int)

# prepare cell groupings dictionary (discrete)
cg = obs.iloc[cell_ix][toappend].astype(str).to_dict(orient='list')

# prepare numerical
ctracks = obs[cell_mask][toappend_cont].fillna(0).astype(float).to_dict(orient='list')

# append categorical
srz.append_cell_groupings(project_dir+plot_name,cg,colordd=cdd)
# append numerical
srz.append_color_tracks(ctracks,project_dir+plot_name,backup=True)

In [28]:
plot_name = 'kidney_v0_156_v2000' #adding to umap as well

In [29]:
# color dictionary
cdd = srz.read_cell_groupings(project_dir+plot_name+'/categorical_coloring_data.json')
cdd = {key:value['label_colors'] for key,value in cdd.items()}

# get cell filter:
cell_ix = np.loadtxt(project_dir+plot_name+'/cell_filter.txt',dtype=int)

# prepare cell groupings dictionary (discrete)
cg = obs.iloc[cell_ix][toappend].astype(str).to_dict(orient='list')

# prepare numerical
ctracks = obs[cell_mask][toappend_cont].fillna(0).astype(float).to_dict(orient='list')

# append categorical
srz.append_cell_groupings(project_dir+plot_name,cg,colordd=cdd)
# append numerical
srz.append_color_tracks(ctracks,project_dir+plot_name,backup=True)